  ## Clustering sur la decennie 

### imports

In [1]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine


In [ ]:
import nltk

nltk.download('punkt')
nltk.download("punkt_tab")

In [3]:
data_path = "../../data/txt/"

## Choisir une décennie

In [4]:
DECADE = '1960'

## Charger tous les  fichiers de la décennie et en créer une liste de textes

In [5]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [ ]:
# Exemple de fichiers
files[:5]

In [7]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

## Vectoriser les documents à l'aide de TF-IDF

In [9]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [10]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Imprimer le vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

## Comprendre les vecteurs et leurs "distances"

In [ ]:
cosine([1, 2, 3], [1, 2, 3])

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
cosine([1, 2, 3], [2, 2, 2])

### Tests sur nos documents

In [17]:
tfidf_array = tfidf_vectors.toarray()

In [ ]:
# Vecteur du document 0
tfidf_array[0]

In [ ]:
# Vecteur du document 1
tfidf_array[1]

In [ ]:
cosine(tfidf_array[0], tfidf_array[1])

## Appliquer un algorithme de clustering sur les vecteurs TF-IDF des documents

### Définir un nombre de clusters

In [88]:
N_CLUSTERS = 3

### Instancier le modèle K-Means et ses arguments

In [89]:
km_model = KMeans(n_clusters=N_CLUSTERS)

### Appliquer le clustering à l'aide de la fonction `fit_predict`

In [91]:
clusters = km_model.fit_predict(tfidf_vectors)

In [92]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import collections
from pprint import pprint




clusters = km_model.fit_predict(tfidf_vectors)

# Obtenez les noms des features (mots) à partir du vectorizer
feature_names = vectorizer.get_feature_names_out()

# Définir le nombre de mots-clés à extraire par cluster
num_keywords = 20

print(f"\n--- Top {num_keywords} mots-clés par cluster (pour K={km_model.n_clusters}) ---")
print("-" * 60)

# Pour chaque cluster identifié par K-Means
for i in range(km_model.n_clusters):
    cluster_tfidf_vectors = tfidf_vectors[clusters == i]
    if cluster_tfidf_vectors.shape[0] > 0:
        cluster_tfidf_mean = cluster_tfidf_vectors.mean(axis=0) 
        top_indices = np.array(cluster_tfidf_mean).flatten().argsort()[::-1][:num_keywords]

        # Récupérer les mots correspondants
        top_words = [feature_names[idx] for idx in top_indices]

        # Obtenir le nombre de documents dans ce cluster
        num_docs_in_cluster = cluster_tfidf_vectors.shape[0]

        print(f"Cluster {i} (Documents: {num_docs_in_cluster}):")
        print(f"  Mots-clés : {', '.join(top_words)}")
        print("-" * 60)
    else:
        print(f"Cluster {i} est vide.")
        print("-" * 60)

## Visualiser les clusters

### Réduire les vecteurs à 2 dimensions à l'aide de l'algorithme PCA
Cette étape est nécessaire afin de visualiser les documents dans un espace 2D

https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales

In [50]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

### Générer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

# Word Embeddings : le modèle Word2Vec

In [53]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [54]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [55]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [56]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [ ]:
len(bigram_phrases.vocab.keys())

Prenons une clé au hasard :

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

Le dictionnaire indique le score de cette coocurrence :

In [ ]:
bigram_phrases.vocab[key_]

In [61]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [62]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [63]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [64]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

## Entrainement d'un modèle Word2Vec sur ce corpus

In [110]:
from nltk.corpus import stopwords
import string
import re

# Stopwords de base en français
stop_words = set(stopwords.words('french'))

# Stopwords personnalisés pour nettoyer les artefacts typographiques et mots peu informatifs
custom_stopwords = {
    '--', '>>', '<<', '...', '«', '»', '’', '``', "''", '.', ',', ';', ':', '!', '?',
    'http', 'https', 'www', 'com', 'fr', 'org', 'là', 'ainsi', 'toutefois',
    'article', 'chapitre', 'figure', 'exemple', 'etc', 'cela', 'celui', 'ceux',
    'très', 'peu', 'dont', 'tandis', 'lorsque', 'entre', 'chaque', 'aucun'
}
stop_words.update(custom_stopwords)

# Ponctuation standard
punctuation = set(string.punctuation)

# Nettoyage du corpus
corpus_cleaned = []
for doc in corpus:
    cleaned_doc = []
    for word in doc:
        word = word.lower().strip()
        if (
            len(word) > 2 and               # Ignore mots trop courts
            not word.isdigit() and         # Ignore chiffres
            word not in stop_words and     # Ignore stopwords
            word not in punctuation and    # Ignore ponctuation
            re.match("^[a-zàâçéèêëîïôûùüÿñæœ-]+$", word)  # Garde uniquement les mots alpha valides
        ):
            cleaned_doc.append(word)
    if cleaned_doc:
        corpus_cleaned.append(cleaned_doc)


In [ ]:
%%time
model = Word2Vec(
    corpus_cleaned,
    vector_size=32,
    window=5,
    min_count=5,
    workers=4,
    epochs=5
)

### Sauver le modèle dans un fichier

In [ ]:
outfile = f"../../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [ ]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [ ]:
model.wv["ministre"]

### Calculer la similarité entre deux termes

In [115]:
model.wv.similarity("ministre", "roi")

0.56965774

### Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("ministre", topn=10)

### Faire des recherches complexes à travers l'espace vectoriel

In [ ]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

In [ ]:
import gensim
from gensim.models import Word2Vec
import logging
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import collections
from pprint import pprint
import numpy as np

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    print("t-SNE et Matplotlib importés avec succès.")
except ImportError:
    print("matplotlib ou scikit-learn non installés. La visualisation ne sera pas disponible.")

# Configuration du logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Chemin vers le modèle Word2Vec
model_path = "../../data/newspapers.model"

# Chargement du modèle Word2Vec
try:
    model = Word2Vec.load(model_path)
    print(f"Modèle Word2Vec chargé depuis '{model_path}'")
except FileNotFoundError:
    print(f"Modèle introuvable à l'emplacement '{model_path}'")
    exit()

# Exploration du modèle
print("\n=== Exploration du Modèle Word2Vec ===")
print(f"Vocabulaire : {len(model.wv.key_to_index)} mots")
print(f"Dimensions des vecteurs : {model.wv.vector_size}")

# Mots les plus fréquents
print("\n--- Mots les plus fréquents ---")
most_frequent_words = list(model.wv.key_to_index.keys())[:20]
pprint(most_frequent_words)

# Exemples de similarité
print("\n--- Exemples de similarité entre mots ---")
similarity_examples = [
    ("guerre", "conflit"),
    ("vietnam", "américain"),
    ("étudiant", "manifestation"),
    ("musique", "artiste"),
    ("ministre", "gouvernement"),
    ("voiture", "ford"),
    ("appartement", "villa"),
    ("travail", "emploi"),
    ("décès", "famille"),
    ("chaise", "astronomie")
]

for word1, word2 in similarity_examples:
    if word1 in model.wv and word2 in model.wv:
        sim = model.wv.similarity(word1, word2)
        print(f"Similarité entre '{word1}' et '{word2}': {sim:.4f}")
    else:
        print(f"Mots absents du vocabulaire : '{word1}', '{word2}'")

# Mots les plus similaires
print("\n--- Mots les plus similaires ---")
most_similar_examples = ["président", "lune", "festival", "crise", "femme"]
for target_word in most_similar_examples:
    if target_word in model.wv:
        print(f"\nMots similaires à '{target_word}':")
        pprint(model.wv.most_similar(target_word, topn=10))
    else:
        print(f"Mot absent du vocabulaire : '{target_word}'")

# Analogies sémantiques
print("\n--- Exemples d'analogies ---")
analogy_examples = [
    (["londres", "france"], ["paris"], "Londres est à France ce que Paris est à X"),
    (["reine", "homme"], ["femme"], "Reine est à Femme ce que Homme est à X"),
    (["guerre", "russe"], ["paix"], "Guerre est à Paix ce que Russe est à X"),
    (["chanteur", "chanson"], ["acteur"], "Chanteur est à Chanson ce que Acteur est à X")
]

for positive_words, negative_words, description in analogy_examples:
    if all(word in model.wv for word in positive_words + negative_words):
        print(f"\nAnalogie : {description}")
        pprint(model.wv.most_similar(positive=positive_words, negative=negative_words, topn=5))
    else:
        missing = [w for w in positive_words + negative_words if w not in model.wv]
        print(f"\nAnalogie '{description}' : mots manquants : {missing}")

# Visualisation avec t-SNE
if 'TSNE' in locals() and 'plt' in locals():
    print("\n--- Visualisation t-SNE ---")
    frequent_words_subset = list(model.wv.key_to_index.keys())[20:120]
    specific_words = [
        "guerre", "paix", "vietnam", "union", "soviétique", "américain", "président", "politique",
        "démocratie", "révolution", "étudiant", "manifestation", "jeunesse", "liberté",
        "droit", "femme", "intégration", "raciale", "mur_de_berlin", "crise",
        "lune", "espace", "cosmonaute", "satellite", "astronaute", "mission",
        "musique", "film", "théâtre", "festival", "artiste", "chanteur", "rock", "pop",
        "marché", "actions", "bourse", "économie", "francs", "dollar", "entreprise",
        "paris", "londres", "bruxelles", "washington", "moscou", "berlin", "rome", "saigon",
        "appartement", "villa", "louer", "vendre", "voiture", "ford", "peugeot", "emploi", "travail", "agence", "décès"
    ]

    words_to_visualize = [w for w in set(frequent_words_subset + specific_words) if w in model.wv]

    if len(words_to_visualize) > 50:
        word_vectors = np.array([model.wv[word] for word in words_to_visualize])
        tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=2500, init='pca', learning_rate='auto')

        vectors_2d = tsne.fit_transform(word_vectors)

        plt.figure(figsize=(15, 15))
        for i, word in enumerate(words_to_visualize):
            plt.scatter(vectors_2d[i, 0], vectors_2d[i, 1], s=10)
            plt.annotate(word, xy=(vectors_2d[i, 0], vectors_2d[i, 1]), xytext=(3, 2),
                         textcoords='offset points', ha='right', va='bottom', fontsize=7)
        plt.title(f"Visualisation t-SNE des embeddings de {len(words_to_visualize)} mots")
        plt.grid(True)
        plt.savefig("tsne_word_embeddings.png", dpi=300, bbox_inches='tight')
        plt.show()
        print("Image sauvegardée : tsne_word_embeddings.png")
    else:
        print("Pas assez de mots pour une visualisation t-SNE significative.")
